In [1]:
import ShapeRetrieval: ShapeRetrieval as SR
using ShapeRetrieval: LearnedTimeDiffusionBlock
import ShapeRetrieval: Mesh, heat_integrator
using LinearAlgebra
using SparseArrays
using Flux
using Zygote

### Visualization Tools

In [2]:
bunny = SR.load_obj("./meshes/gourd.obj")
bunny = SR.normalize_mesh(bunny)
println("nv: $(bunny.nv) nf: $(bunny.nf) Area: ", sum(bunny.vertex_area))
println(count(x->x != 0, bunny.cot_laplacian) / (bunny.nv ^2 )) 

nv: 326 nf: 648 Area: 2.681867885523121
0.02135947909217509

In [3]:
function spectral_loss(model, x, λ, ϕ, A, y) 
    norm(model(x, λ, ϕ, A) - y)
end

spectral_loss (generic function with 1 method)

## Example Prediction using Spectral Mode

In [ ]:
m = LearnedTimeDiffusionBlock(2, :implicit)
heat_signal = zeros(bunny.nv,2)
heat_signal[1, 1] = 1.0
heat_signal[200,2] = 1.0
# λ, ϕ, A = SR.get_diffusion_inputs(bunny, :spectral)
L, A = SR.get_diffusion_inputs(bunny, :implicit)
m(heat_signal, L, M, A)

In [12]:
println(sum(bunny.face_area))
rescaled_bunny = SR.normalized_area_mesh(bunny)
println(sum(rescaled_bunny.face_area))

2.681867885523122


UndefVarError: UndefVarError: `normalized_area_mesh` not defined

In [19]:
v_sphere, f_sphere = SR.get_in_sphere(bunny, 3, 0.1)
v_reach, f_reach = SR.connected_mesh_in_sphere(bunny, 3, 0.1)
.
# should be false. check that connection doesn't add any additional verts/faces
println(any(.!v_sphere .& v_reach))
println(any(.!f_sphere .& f_reach))

false
false


In [8]:
centered = SR.rotate_mesh_random(bunny)

Mesh([0.1677752444808875 0.1449528206873632 … -0.17767480347370457 -0.21066262640187042; 0.2732483906038117 0.279924851886468 … 0.4536473865910472 0.48378868943111364; 0.6067453476426498 0.583924779856867 … 0.7747520412639837 0.7580179783073167], [1 4 … 325 324; 2 5 … 324 325; 3 3 … 319 326], [0.3899641976892244 0.16196094505624783 … -0.2920742348545539 -0.6999941254522594; 0.9125769108875599 0.9782702849750473 … -0.9240516825487431 -0.7058564582827184; -0.12300937458388395 -0.12944458973370324 … -0.24661940173669164 0.10851214048623227], [0.3899641976892244 0.16196094505624783 … -0.2920742348545539 -0.6999941254522594; 0.9125769108875599 0.9782702849750473 … -0.9240516825487431 -0.7058564582827184; -0.12300937458388395 -0.12944458973370324 … -0.24661940173669164 0.10851214048623227], [2.6911162589581203 1.2769359939173786 … -1.7472162138675884 -1.5159845488670938; 4.130954925971927 4.645911776870183 … -2.0539690017459877 1.4747833076336194; -0.421053787844362 -0.8748257170184637 … 1.8

#### Sample Training 

In [17]:
λ, ϕ, A = SR.get_diffusion_inputs(bunny, :implicit)

opt_state = Flux.setup(Adam(), m);

y_true = m(heat_signal, λ, ϕ, A)
println("t_init ", m.diffusion_time)
m = LearnedTimeDiffusionBlock(2, :implicit)
println("t_start: ", m.diffusion_time)
println("start cost ", spectral_loss(m, heat_signal, λ, ϕ, A, y_true))
@time for i=1:2000
    x, y  = heat_signal, y_true
    grad = gradient(spectral_loss, m, x, λ, ϕ, A, y)
    Flux.update!(opt_state, m, grad[1])
end
println(spectral_loss(m,heat_signal, λ, ϕ, A, y_true))
println(m)

MethodError: MethodError: no method matching get_diffusion_inputs(::Mesh, ::Symbol)

Closest candidates are:
  get_diffusion_inputs(::Mesh, !Matched::Type{<:ShapeRetrieval.DiffusionMode})
   @ ShapeRetrieval ~/Documents/MIT/_S2023/ShapeRetrieval/src/utils.jl:31


In [6]:
m(heat_signal, SR.get_diffusion_inputs(bunny)...)
println(m.diffusion_time)

(200, 2)


[2.9274950637401447, 0.7993044171182087]


### AD with Implicit Mode

In [7]:
m = LearnedTimeDiffusionBlock(2, :implicit)
Flux.trainable(m::LearnedTimeDiffusionBlock) = (m.diffusion_time,)

function implicit_loss(model, x, L, M, A::Vector{Float64}, y)
    norm(model(x, L, M, A) - y)
end

inputs = SR.get_diffusion_inputs(bunny,:implicit)
m(heat_signal, inputs...)

0.30148325561767775

In [49]:
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
L, M, A = SR.get_diffusion_inputs(bunny, :implicit)
display(m(heat_signal, L, M, A))
data = [(heat_signal, bunny, 0.0),]

opt_state = Flux.setup(Adam(), m)

for i=1:1000
    for d in data
        x, mesh, y  = d
        grad = gradient(implicit_loss, m, x, L,M,A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end

display(m(heat_signal, L, M, A))

0.3014815281221147

0.301481517244275